### Instalando o modelo Whisper da OpenAI e executando o modelo (isso foi feito no Google Colab)

In [ ]:
%pip install git+https://github.com/openai/whisper.git
!sudo apt update && sudo apt install ffmpeg

In [ ]:
!whisper "audio.mp4" --model medium --language pt

### Agora, vamos fazer isso criando um "cliente" para o modelo Whisper da OpenAI. Para isso, vamos usar a biblioteca OpenAI e a Langchain.

In [ ]:
import os

from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from openai import OpenAI
from pydub import AudioSegment

In [ ]:
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

audio = AudioSegment.from_file("/content/audio.mp4", format="mp4")
audio.export("/content/audio.wav", format="wav")

client = OpenAI(api_key=OPENAI_API_KEY)

audio_file = open("/content/audio.wav", "rb")
transcription = client.audio.transcriptions.create(model="whisper-1", file=audio_file)

print(transcription.text)

In [ ]:
client = OpenAI(api_key=OPENAI_API_KEY)

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {
            "role": "system",
            "content": """
    Você é um assistente especializado em tarefas de resposta a perguntas voltadas a um hospital.
    Você tem a capacidade de entender e compreender a dúvida de uma pessoa que ligou
    para os serviços do hospital. Se você não souber a resposta, diga apenas que vai repassar a
    ligação para um atendente humano.
    Responda a pergunta do usuário.
    """,
        },
        {
            "role": "user",
            "content": "Preciso de um pronto socorro, para uma pessoa que foi baleada, onde posso ir?",
        },
        {
            "role": "assistant",
            "content": """Olá, você pode ir até o O'Neil Hospital
    que fica localizado na Rua Orlow, Quadra 06, Lote 04, Número 1943.""",
        },
        {"role": "user", "content": f"{transcription.text}"},
    ],
)

print(response.choices[0].message.content)